In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mediapipe
!pip install --upgrade scikit-learn

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
DATA_PATH = os.path.join('/kaggle/input/mpdata/MP_DATA')

actions = np.array(['a','b','c','d','e','f','g',
                    'h','i','j','k','l','m','n',
                    'o','p','q','r','s','t','u',
                    'v','w','x','y','z'])

#no of videos
no_sequnces = 30

#number of frames
sequnce_length = 30 


In [ ]:
DATA_PATH = os.path.join('/kaggle/input/wlpsl/WLPSL')
action = []
for subdir,dirs,files in os.walk(DATA_PATH +"/Videos"):
    for dirc in dirs:
        action.append(dirc)

actions = np.array(action)

#no of videos
no_sequnces = 30

#number of frames
sequnce_length = 30 


In [ ]:
actions

In [ ]:
len(actions)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num,label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences,labels = [],[]
for action in actions:
    for sequence in range(no_sequnces):
        window = []
        for frame_num in range(sequnce_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.unique(labels)

In [ ]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05)

In [ ]:
print(x.shape)   # should be (num_samples, 30, 1662) or similar
print(y.shape)   # should be (num_samples,)
print(np.unique(np.argmax(y, axis=1)))  # should show 26 unique numbers (0–25)


In [ ]:
import joblib

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train = X_train_scaled.reshape(X_train.shape)


In [ ]:

X_test_reshaped = x_test.reshape(-1, x_test.shape[-1])
X_test_scaled = scaler.transform(X_test_reshaped)
x_test = X_test_scaled.reshape(x_test.shape)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(700)
X_train_reduced = pca.fit_transform(X_train.reshape(-1, X_train.shape[-1]))
X_train_reduced = X_train_reduced.reshape(X_train.shape[0], X_train.shape[1], 700)


In [ ]:
X_test_reduced = pca.transform(x_test.reshape(-1, x_test.shape[-1]))
X_test_reduced = X_test_reduced.reshape(x_test.shape[0], x_test.shape[1], 700)


In [ ]:
import joblib
joblib.dump(scaler,"scalerV2.pkl")
joblib.dump(pca,"pcaV2_700.pkl")


In [ ]:
import numpy as np
import random

def augment_sequence(sequence, noise_std=0.001, dropout_prob=0.1):
    """
    sequence: np.array of shape (30, 1662)
    """
    seq = sequence.copy()

    # 1. Add small Gaussian noise
    seq += np.random.normal(0, noise_std, seq.shape)

    # 2. Random feature scaling
    seq *= np.random.uniform(0.95, 1.02)

    # 3. Random frame dropout
    if random.random() < dropout_prob:
        drop_idx = np.random.choice(seq.shape[0], size=1, replace=False)
        seq = np.delete(seq, drop_idx, axis=0)
        # Pad back to 30 frames (duplicate last frame)
        seq = np.pad(seq, ((0, 1), (0, 0)), mode='edge')

    return seq


In [ ]:
X_augmented = []
y_augmented = []

for x, y in zip(X_train_reduced, y_train):
    X_augmented.append(x)
    y_augmented.append(y)

    # Add one augmented version of each sequence
    X_augmented.append(augment_sequence(x))
    y_augmented.append(y)

X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)


In [ ]:
X_augmented.shape

In [ ]:
y_augmented.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,BatchNormalization,Bidirectional
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2


log_dir = os.path.join('logs')

tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=40,              # stop if val_loss doesn't improve for 20 epochs
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=15,
    min_lr=1e-6,
    verbose=1
)


In [ ]:
X_train_reduced.shape

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(64,return_sequences=True,activation="tanh",input_shape=(30,700))))
model.add(BatchNormalization())
model.add(Dropout(0.2))



model.add(Bidirectional(LSTM(64,return_sequences=False,activation="tanh")))
model.add(Dropout(0.2))


model.add(Dense(128,activation='relu',kernel_regularizer=l2(5e-6)))
model.add(Dropout(0.15))

model.add(Dense(64,activation='relu',))
model.add(Dropout(0.15))

model.add(Dense(32,activation='relu',))
model.add(Dropout(0.1))

model.add(Dense(actions.shape[0],activation='softmax'))





In [ ]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=['accuracy'])


In [ ]:
model.fit(X_augmented,y_augmented,validation_split=0.2,epochs=2000,callbacks=[tb_callback,early_stop,checkpoint,reduce_lr])

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
loss,acc = model.evaluate(X_test_reduced,y_test)

In [ ]:
print(acc)

In [ ]:
model.save("model_12.keras")

In [ ]:
model = load_model("model_11.keras")

# Base CNN Model

In [ ]:
import cv2,os

In [ ]:
DATA_PATH = os.path.join('/kaggle/input/asl-dataset/asl_dataset/asl_dataset')
output_path = "/kaggle/working/outputs"
FRAMES_PER_VIDEO = 30
FRAME_SIZE = (224,224)
OUTPUT_PATH = "/kaggle/working/outputs"

In [ ]:
os.listdir(DATA_PATH)

In [ ]:
import cv2, os

def extract_frames(video_path, output_folder, frame_count=60):  
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print(f"Skipping (no frames): {video_path}")
        return 
    
    # FIXED evenly spaced frames
    frame_indices = [int(i * total_frames / frame_count) for i in range(frame_count)]
    frame_indices = sorted(set(frame_indices))

    saved = 0
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if count in frame_indices:
            frame = cv2.resize(frame, FRAME_SIZE)
            frame_name = f"frame_{saved}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_name), frame)
            saved += 1
        
        count += 1

        if saved >= len(frame_indices):
            break

    cap.release()
    print(f"Extracted {saved} frames from {video_path}")


In [ ]:
def extract_all_frames():
    for class_name in os.listdir(DATA_PATH):
        class_path = os.path.join(DATA_PATH, class_name)

        if not os.path.isdir(class_path):
            continue

        print(f"\nProcessing Class: {class_name}")

        output_dir = os.path.join(output_path, class_name)
        os.makedirs(output_dir, exist_ok=True)

        for video_file in os.listdir(class_path):
            if video_file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):

                video_path = os.path.join(class_path, video_file)

                video_output_folder = os.path.join(
                    output_dir,
                    video_file.split('.')[0]
                )
                os.makedirs(video_output_folder, exist_ok=True)

                print(f"Extracting ALL frames from: {video_file}")
                extract_frames(video_path, video_output_folder)

    print("\nAll Frames Extracted Successfully")


In [ ]:
extract_all_frames()

In [ ]:
def preprocess_frame(frame_path,dataset_dir,frame):
    os.makedirs(dataset_dir,exist_ok=True)
    
    img = cv2.imread(frame_path)
    resized_img = cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)
    rgb_img = cv2.cvtColor(resized_img,cv2.COLOR_BGR2RGB)
    
    norm_img = cv2.normalize(
    rgb_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
     
    final_img = norm_img
    bgr_final = cv2.cvtColor(final_img, cv2.COLOR_RGB2BGR)
    folder_path = os.path.join(dataset_dir)
    cv2.imwrite(os.path.join(folder_path,frame),bgr_final)

    


In [ ]:
preprocess_output_path = "/kaggle/working/new_dataset_asl"
alphabetinput = '/kaggle/input/dataset-pakistani-sign-language/dataset'
os.makedirs(preprocess_output_path,exist_ok=True)

def process_all_frames():
    for class_name in os.listdir(DATA_PATH):
        class_path = os.path.join(DATA_PATH,class_name)
        
        if not os.path.isdir(class_path):
            continue
    
        for video_num in os.listdir(class_path):
            frame_dir = os.path.join(class_path,video_num)
    
            for frame in os.listdir(frame_dir):
                
                frame_path = os.path.join(frame_dir,frame)
                
                dataset_path = os.path.join(preprocess_output_path,class_name)
                dataset_dir = os.path.join(dataset_path,video_num)
                print(dataset_dir)
                print(f"Preprocessing {frame} of {class_name}")
                preprocess_frame(frame_path,dataset_dir,frame)
                
                
    


In [ ]:
#for alphabets
def process_all_frames():
    for class_name in os.listdir(DATA_PATH):
        class_path = os.path.join(DATA_PATH, class_name)

        if not os.path.isdir(class_path):
            continue

        # output folder for this class
        output_class_dir = os.path.join(preprocess_output_path, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        # iterate all images
        for frame in os.listdir(class_path):
            frame_path = os.path.join(class_path, frame)

            if not frame.lower().endswith((".jpg", ".jpeg", ".png")):
                continue  # skip non-images
            
            print(f"Preprocessing {frame} of {class_name}")
            preprocess_frame(frame_path, output_class_dir, frame)


In [ ]:
process_all_frames()


In [7]:
import tensorflow as tf

In [ ]:
DATASET_PATH = "/kaggle/working/new_dataset_asl"

In [ ]:
import os
os.listdir(DATASET_PATH)

In [ ]:
batch_size = 32
img_size= (224,224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


In [ ]:
BIGGER_ASL_DATASET = "/kaggle/input/american-sign-language/ASL_Dataset"

In [ ]:
train = os.path.join(BIGGER_ASL_DATASET,"Train")
test = os.path.join(BIGGER_ASL_DATASET,"Test")

In [ ]:
import tensorflow as tf
batch_size = 32
img_size= (224,224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test,
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


In [ ]:
class_names = train_ds.class_names
print(class_names)

for images,labels in train_ds.take(1):
    print(images.shape)
    print(labels.shape)

In [ ]:
len(class_names)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))



train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

for images, labels in train_ds.take(1):
    print(f"After normalization - Min: {images.numpy().min():.3f}, Max: {images.numpy().max():.3f}, Mean: {images.numpy().mean():.3f}")
    # Should now print: Min: 0.0, Max: 1.0 (approximately)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Rescaling



In [ ]:
num_classes=len(class_names)

model = Sequential([
    
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Dropout(0.2),
    
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.3),

    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.3),

    Flatten(),

    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(num_classes,activation='softmax')
    
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor='val_loss',           # Monitor validation loss
    patience=3,                    # Stop if no improvement for 5 epochs
    restore_best_weights=True,     # Restore weights from best epoch
    verbose=1                      # Print messages
)

# Optional: Save the best model during training
model_checkpoint = ModelCheckpoint(
    'best_model.keras',           # or 'best_model.h5'
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]

)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax[0].plot(history.history['accuracy'], label='Train Acc')
ax[0].plot(history.history['val_accuracy'], label='Val Acc')
ax[0].set_title("Accuracy")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")
ax[0].legend()
ax[0].grid(True)

# Loss
ax[1].plot(history.history['loss'], label='Train Loss')
ax[1].plot(history.history['val_loss'], label='Val Loss')
ax[1].set_title("Loss")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")
ax[1].legend()
ax[1].grid(True)

plt.show()
plt.savefig("signbuddy_cnn.png",dpi=300,bbox_inches='tight')

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = []
y_pred = []

for batch_x, batch_y in test_ds:
    # predict on this batch
    batch_preds = model.predict(batch_x, verbose=0)
    batch_pred_classes = np.argmax(batch_preds, axis=1)

    # store
    y_true.extend(batch_y.numpy())
    y_pred.extend(batch_pred_classes)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_true, y_pred)
print(cm)

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Get one batch for visualization
for images, labels in test_ds.take(1):
    preds = model.predict(images)
    pred_classes = np.argmax(preds, axis=1)
    
    plt.figure(figsize=(15, 10))
    for i in range(min(9, len(images))):
        plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy())
        plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[pred_classes[i]]}")
        plt.axis('off')
    plt.tight_layout()

    plt.show()

In [ ]:
model.save("signbuddy_cnn_asl_model_4.h5")
model.save("signbuddy_cnn_asl_model_4.keras")

In [1]:
from tensorflow.keras.models import load_model

2025-12-01 05:45:23.576441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764567923.819990      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764567923.889228      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = load_model('/kaggle/working/signbuddy_cnn_asl_model_4.keras')

2025-12-01 05:45:40.214198: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
# Evaluate on the SAME test set that gave you 100% accuracy
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc}")

# Get predictions for the entire test set
predictions = []
true_labels = []

for images, labels in test_ds:
    preds = model.predict(images, verbose=0)
    predictions.extend(preds.argmax(axis=1))
    true_labels.extend(labels.numpy())

from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions, target_names=class_names))

I0000 00:00:1764568175.824789      89 service.cc:148] XLA service 0x7c89c4008870 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764568175.825651      89 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1764568176.317569      89 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 755ms/step - accuracy: 1.0000 - loss: 0.0035
Test Accuracy: 1.0
              precision    recall  f1-score   support

           A       1.00      1.00      1.00         4
           B       1.00      1.00      1.00         4
           C       1.00      1.00      1.00         4
           D       1.00      1.00      1.00         4
           E       1.00      1.00      1.00         4
           F       1.00      1.00      1.00         4
           G       1.00      1.00      1.00         4
           H       1.00      1.00      1.00         4
           I       1.00      1.00      1.00         4
           J       1.00      1.00      1.00         4
           K       1.00      1.00      1.00         4
           L       1.00      1.00      1.00         4
           M       1.00      1.00      1.00         4
           N       1.00      1.00      1.00         4
     Nothing       1.00      1.00      1.00         4
           O       1.00      1.00      1.00

In [18]:
import tensorflow as tf



converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("asl_alphabert_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmptqv7gko8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_15')
Output Type:
  TensorSpec(shape=(None, 28), dtype=tf.float32, name=None)
Captures:
  136932680043984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680046288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680045712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680044752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680048400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680049168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680049360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680050896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680051088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136932680052816: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1764568217.049429      38 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1764568217.049472      38 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
